In [0]:
import torch
import torchvision
import torchvision.datasets
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import pandas as pd
import importlib
from torch.utils.data.sampler import SubsetRandomSampler

from IPython.core.ultratb import AutoFormattedTB

__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

assert torch.cuda.is_available(), "You need to request a GPU from Runtime > Change Runtime"

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
# !curl -OL https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
!cp /content/gdrive/My\ Drive/SimCLR/data/stanfordCars/car_devkit.tgz .
# !curl -OL http://imagenet.stanford.edu/internal/car196/cars_train.tgz
!cp /content/gdrive/My\ Drive/SimCLR/data/stanfordCars/cars_train.tgz .
# !curl -OL http://imagenet.stanford.edu/internal/car196/cars_test.tgz
!cp /content/gdrive/My\ Drive/SimCLR/data/stanfordCars/cars_test.tgz .
!tar xvf car_devkit.tgz
!tar xf cars_train.tgz
!tar xf cars_test.tgz

devkit/
devkit/cars_meta.mat
devkit/cars_train_annos.mat
devkit/cars_test_annos.mat
devkit/README.txt
devkit/train_perfect_preds.txt
devkit/eval_train.m


In [5]:
!rm -rf CS601R_Final_Project/ models/ datasets/ notebooks/
!git clone https://github.com/webMan1/CS601R_Final_Project.git
!cp -r CS601R_Final_Project/* .

Cloning into 'CS601R_Final_Project'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 99 (delta 49), reused 72 (delta 22), pack-reused 0
Unpacking objects: 100% (99/99), done.


In [0]:
import datasets.StanfordCarsDataset
import models.CLRDecoder
import importlib

datasets.StanfordCarsDataset = importlib.reload(datasets.StanfordCarsDataset)
models.CLRDecoder = importlib.reload(models.CLRDecoder)

CLRDecoder = models.CLRDecoder.CLRDecoder

In [0]:
class CarsEndToEndBaseline(nn.Module):
  def __init__(self):
    super(CarsEndToEndBaseline, self).__init__()
    resnet = torchvision.models.resnet50(pretrained=False)
    self.encoder = nn.Sequential(*list(resnet.children())[:-1])
    self.clr = CLRDecoder(196, True)
  
  def forward(self, x):
    encoded = self.encoder(x).view((-1, 2048))
    decoded = self.clr(encoded)
    return decoded

# Training Loop

In [0]:
def train(model, optimizer, train_loader, valid_loader, num_epochs:int, valid_freq:int=10):
  train_len = len(train_loader)
  valid_len = len(valid_loader)
  loop = tqdm(total=(num_epochs * train_len + (num_epochs // valid_freq) * valid_len), position=0)

  train_losses = []
  train_accs = []

  valid_losses = []
  valid_accs = []

  criteria = nn.CrossEntropyLoss()

  for e in range(num_epochs):
    loss_builder = []
    acc_builder = []

    for i, (x, y_truth) in enumerate(train_loader):
      x, y_truth = x.cuda(async=False), y_truth.long().squeeze(1).cuda(async=False)
      optimizer.zero_grad()
      
      y_hat = model(x)

      total_loss = criteria(y_hat, y_truth)
      total_loss.backward()

      acc = y_truth.eq(y_hat.argmax(1)).sum().item() / len(y_truth)

      optimizer.step()

      loss_builder.append(total_loss.item())
      acc_builder.append(acc)

      loop.update(1)
      loop.set_description(f"Epoch: {e}, it: {i}/{train_len}. Loss: {total_loss.item()}. Acc: {acc}")
    
    train_accs.append(acc_builder)
    train_losses.append(loss_builder)

    if e % valid_freq == 0:
      loss_builder = []
      acc_builder = []

      with torch.no_grad():
        model.eval()

        for i, (x, y_truth) in enumerate(valid_loader):
          x, y_truth = x.cuda(async=False), y_truth.long().squeeze(1).cuda(async=False)

        y_hat = model(x)

        total_loss = criteria(y_hat, y_truth)
        acc = y_truth.eq(y_hat.argmax(1)).sum().item() / len(y_truth)

        loss_builder.append(total_loss.item())
        acc_builder.append(acc)

        loop.update(1)
        loop.set_description(f"[VALIDATING] Epoch: {e}, it: {i}/{valid_len}. Loss: {total_loss.item()}. Acc: {acc}")

      model.train()

      valid_accs.append(acc_builder)
      valid_losses.append(loss_builder)
    
    state = {
        "model": model.state_dict(),
        "train_losses": train_losses,
        "train_accs": train_accs,
        "valid_losses": valid_losses,
        "valid_accs": valid_accs,
        "epoch": e
    }
    num = ((e + 4) // 5) * 5
    path = f"/content/gdrive/My Drive/SimCLR/models/stanford/e2e_e_{num}_linear.mod"
    torch.save(state, path)

  return train_losses, train_accs, valid_losses, valid_accs

In [0]:
def run_training():
  batch_size = 32
  model = CarsEndToEndBaseline().cuda()
  optimizer = optim.Adam(model.parameters())

  train_loader = datasets.StanfordCarsDataset.get_loader(True, batch_size)
  valid_loader = datasets.StanfordCarsDataset.get_loader(False, batch_size)

  num_epochs = 10
  
  train_losses, train_accs, valid_losses, valid_accs = train(model, optimizer, train_loader, valid_loader, num_epochs=num_epochs, valid_freq=5)

  return train_losses, train_accs, valid_losses, valid_accs

In [0]:
train_losses, train_accs, valid_losses, valid_accs = run_training()

Epoch: 0, it: 254/255. Loss: 5.283675193786621. Acc: 0.0:   8%|▊         | 255/3054 [03:29<36:37,  1.27it/s]

In [0]:
!ls content/car_ims

ls: cannot access 'content/car_ims': No such file or directory
